In [2]:
import pandas as pd

In [3]:
f = open("program.asm",'r')
lines = f.read()

In [4]:
lines

'start:\n    mvi b, 0x00\n    mvi c, 0x07\n    \nbit: \n    in 0x02\n    ani 0x80\n    ora b\n    rar\n    MOV b, a\n    mvi a, 0x73\n\ndelay:\n    dcr a\n    jnz delay\n    dcr c\n    jnz bit\n    in 0x02\n    ani 0x80\n    ora b\n    sta 0x2100\n\n\n\n'

In [5]:
class BruhFoundException(Exception):
    def __init__(self, line_no, line, message):
        super().__init__(message)
    

In [6]:
program_lines = []
label = None
for i, line in enumerate(lines.split("\n")):
    # check for label
    if line.__contains__(":"):
        label = line.split(":")[0].strip()
        continue
    
    program_lines.append({"label":label,"raw_line":line.strip()})
    label = None
        

In [7]:
pd.DataFrame(program_lines)

,label,raw_line
0,start,"mvi b, 0x00"
1,None,"mvi c, 0x07"
2,None,
3,bit,in 0x02
4,None,ani 0x80
5,None,ora b
6,None,rar
7,None,"MOV b, a"
8,None,"mvi a, 0x73"
9,None,


In [8]:
for line in lines.split("\n"):
    print(line)

start:
    mvi b, 0x00
    mvi c, 0x07
    
bit: 
    in 0x02
    ani 0x80
    ora b
    rar
    MOV b, a
    mvi a, 0x73

delay:
    dcr a
    jnz delay
    dcr c
    jnz bit
    in 0x02
    ani 0x80
    ora b
    sta 0x2100






# construct symbol table

In [9]:
symbol_table_path = "/home/raufie/work/study/computer_engineering/sap2/instruction_set_map.txt"

# Allocate addresses

In [53]:
line = program_lines[0]['raw_line']

In [107]:
line = 'mvib, 235'

In [133]:
def parse_arg_as_hex(arg:str, arg_len:str):
    length = len(arg_len.split('x')[1])
    if arg.__contains__("x"):
        return '0x'+arg.split("x")[1].zfill(length)
    elif arg.__contains__("b"):
        n_str = arg.split("b")[1]
        return '0x'+str(hex(int(n_str, base=2)))[2:].zfill(length)
    else:
        return '0x'+str(hex(int(arg, base=10)))[2:].zfill(length)  

In [154]:
from assembler import Assembler

In [155]:
assembler = Assembler(symbol_table_path)

In [158]:
symbol_table = assembler.symbol_table

In [208]:
def construct_namespace(program_lines):
    namespace = {}
    count = 0
    for line in program_lines:
        label = line['label']
        raw_line = line['raw_line'].lower()
        if len(raw_line.strip())==0:
            continue

        
        comps = raw_line.split(" ") 
        agg = ""
        is_op_found = False
        for i, component in enumerate(comps):
            agg+=component
            if agg in symbol_table:
                if label is not None:
                    namespace[label] = parse_arg_as_hex(str(count),'0xHHHH')
                count+= symbol_table[agg]['count']
                is_op_found = True            
                

        if not is_op_found:
            print(comps)
            raise Exception("SyntaxError: Can't find the op") 
        
    return namespace
            

In [215]:
namespace = construct_namespace(program_lines)

In [247]:
def translate_line(line_no, line, symbol_table, namespace):
    comps = line.split(" ") 
    agg = ""
    for i, component in enumerate(comps):
        agg+=component
        if agg in symbol_table:
            arg = None
            if symbol_table[agg]['arg'] is not None:
                if len(comps) < i+2:
                    raise Exception(f"Line-{line_no}: {symbol_table[agg]['base']} expected an Argument type {symbol_table[agg]['arg']}\nPlease use the template {symbol_table[agg]['full_mnemonic']}")    
                arg = comps[i+1]
                if arg in namespace:
                    arg = namespace[arg]
                else:
                    arg = parse_arg_as_hex(arg, symbol_table[agg]['arg'])
                if len(arg)!=len(symbol_table[agg]['arg']):
                    raise Exception(f"Line-{line_no}: Length of argument in hex is {len(arg)-2}, but it should be {len(symbol_table[agg]['arg'])-2}, please make sure its not too large")
            return symbol_table[agg], arg
    raise Exception(f"Line-{line_no}: SyntaxError: Can't find the op\n {line}")

In [ ]:
for i, program_line in enumerate(program_lines):
    if len(program_line['raw_line'].strip()) == 0:
        continue
    translate_line(i+1,program_line['raw_line'].lower(), symbol_table, namespace)


In [234]:
translate_line('out 0x03')

({'op_code': '0xd3',
  'count': 1,
  'key': 'out0x03',
  'arg': None,
  'base': 'out',
  'full_mnemonic': 'out 0x03 '},
 None)

In [124]:
mnemonic_arr = line.split(" ")
try:
    mnemonic_arr.remove('')
except:
    pass
mnemonic_key = "".join(mnemonic_arr)
print("".join(mnemonic_arr), end=' ')


mvib,235 

# get the correct op code

In [17]:
symbol_table['lda0x9234']

KeyError: 'lda0x9234'

# Testing

In [1]:
from assembler import Assembler

In [2]:
assembler = Assembler("/home/raufie/work/study/computer_engineering/sap2/instruction_set_map.txt")

In [3]:
assembler.assemble("/home/raufie/work/study/computer_engineering/sap2/assembler/program.asm")

06000e07db02e680b01f473e733dc20d000dc20400db02e680b0320021

In [4]:
f = open('bruh.hex', 'wb')
for item, arg in translated:
    f.write(bytes.fromhex(item['op_code'][2:]))
    if arg is not None:
        f.write(bytes.fromhex(arg[2:]))
f.close()

NameError: name 'translated' is not defined

In [ ]:
for 

b'\xff'